In [44]:
#Defining the Fraud Engine

class FraudDetectionEngine:
    def __init__(self):
        self.rules = {
            'Stop location does not match scheduled location!': self.rule_stop_location_matches_scheduled,
            'Location data is switch off!': self.rule_location_data_off,
            'Data updated for customer at a different location than the scheduled point!': self.rule_application_updates_mismatch,
            'Stop at a Delinquent location!': self.rule_delinquent_location_visited,
            'Payment not collected from customer!': self.rule_uncollected_payments,
            'Pattern Detected- Repeated PTP requests!': self.rule_repeated_ptp_reports
        }

    def evaluate_rules(self, data):
        alerts = []
        for rule_name, rule_function in self.rules.items():
            if rule_function(data):
                alerts.append(f"ALERT: {rule_name}")
        return alerts
    
    #Defining the Rules
    
    #Does the stop location match with scheduled collection location?
    def rule_stop_location_matches_scheduled(self, data):
        stops = data['stops']
        scheduled_locations = data['scheduled_locations']
        for stop in stops:
            if stop['location'] not in scheduled_locations:
                return True
        return False
    
    #Was the location data switched off deliberately?
    def rule_location_data_off(self, data):
        stops = data['stops']
        for stop in stops:
            if stop['location_data'] == 'off':
                return True
        return False
    
    #Was data updated for customer at a different location than the scheduled point?
    def rule_application_updates_mismatch(self, data):
        stops = data['stops']
        application_updates = data['application_updates']
        for stop in stops:
            if stop['location'] in application_updates:
                return False
        return True
    
    #Does the stop location match with recent delinquent or closed customer locations?
    def rule_delinquent_location_visited(self, data):
        stops = data['stops']
        delinquent_locations = data['delinquent_locations']
        for stop in stops:
            if stop['location'] in delinquent_locations:
                return True
        return False
    
    #Was the payment collected from the customer?
    def rule_uncollected_payments(self, data):
        payments = data['payments']
        for payment in payments:
            if not payment['collected']:
                return True
        return False
    
    #Is there a certain pattern in reporting PTP by the field officer?
    def rule_repeated_ptp_reports(self, data):
        reports = data['reports']
        ptp_count = 0
        for report in reports:
            if 'PTP reported' in report['report']:
                ptp_count += 1
        if ptp_count > 3:  # Change this threshold as needed
            return True
        return False



In [45]:
# Sample Data : ALL ALERTS 

# Sample stops data
#This represents the stops made by field officers. 
#Each stop has a location and a status indicating if the location data was switched on or off.
stops = [
    {'location': 'Location A', 'location_data': 'on'},
    {'location': 'Location B', 'location_data': 'off'},  # Location data switched off
]

# Sample scheduled locations
#This is a list of locations where field officers are supposed to visit.
scheduled_locations = ['Location A', 'Location C']

# Sample application updates
#This is a list of locations where updates have been made in the application.
#Location A: Location with updates in the application.
#Location D: Another location with updates, not necessarily visited by the field officers.
application_updates = ['Location A', 'Location D']

# Sample delinquent locations
#This represents locations associated with delinquent or closed customers, which might be flagged for potential fraud.
delinquent_locations = ['Location E', 'Location B']

# Sample payments data
#This is a list of payment records indicating whether payments were collected from customers.
payments = [
    {'customer_id': 1, 'collected': True},
    {'customer_id': 2, 'collected': False},  # Payment not collected
]

# Sample reports data
#This data represents reports made by field officers, specifically looking for patterns in reporting PTP (Promise to Pay).
reports = [
    {'officer_id': 1, 'report': 'PTP reported'},
    {'officer_id': 1, 'report': 'PTP reported'},
    {'officer_id': 1, 'report': 'PTP reported'},
    {'officer_id': 1, 'report': 'PTP reported'},  # Pattern detected
]

# Combine data
data = {
    'stops': stops,
    'scheduled_locations': scheduled_locations,
    'application_updates': application_updates,
    'delinquent_locations': delinquent_locations,
    'payments': payments,
    'reports': reports
}

engine = FraudDetectionEngine()

#Evaluate the rules
alerts = engine.evaluate_rules(data)
for alert in alerts:
    print(alert)


ALERT: Stop location does not match scheduled location!
ALERT: Location data is switch off!
ALERT: Stop at a Delinquent location!
ALERT: Payment not collected from customer!
ALERT: Pattern Detected- Repeated PTP requests!


In [46]:
# Sample Data : NO ALERTS 

stops = [
    {'location': 'Location A', 'location_data': 'on'},
    {'location': 'Location C', 'location_data': 'on'}
]

scheduled_locations = ['Location A', 'Location C']
application_updates = ['Location A', 'Location C']
delinquent_locations = ['Location E', 'Location F']
payments = [
    {'customer_id': 1, 'collected': True},
    {'customer_id': 2, 'collected': True}
]
reports = [
    {'officer_id': 1, 'report': 'PTP reported'},
    {'officer_id': 1, 'report': 'No issues reported'}
]

data = {
    'stops': stops,
    'scheduled_locations': scheduled_locations,
    'application_updates': application_updates,
    'delinquent_locations': delinquent_locations,
    'payments': payments,
    'reports': reports
}

engine = FraudDetectionEngine()

#Evaluate the rules
alerts = engine.evaluate_rules(data)
for alert in alerts:
    print(alert)

In [47]:
#INDIVIDUAL RULES TEST
stops = [
    {'location': 'Location A', 'location_data': 'on'},
    {'location': 'Location B', 'location_data': 'on'}
]

scheduled_locations = ['Location A', 'Location B']
application_updates = ['Location A', 'Location B']
delinquent_locations = ['Location E', 'Location B']
payments = [
    {'customer_id': 1, 'collected': True},
    {'customer_id': 2, 'collected': True}
]
reports = [
    {'officer_id': 1, 'report': 'PTP reported'},
    {'officer_id': 1, 'report': 'No issues reported'}
]

data = {
    'stops': stops,
    'scheduled_locations': scheduled_locations,
    'application_updates': application_updates,
    'delinquent_locations': delinquent_locations,
    'payments': payments,
    'reports': reports
}

engine = FraudDetectionEngine()

#Evaluate the rules
alerts = engine.evaluate_rules(data)
for alert in alerts:
    print(alert)

ALERT: Stop at a Delinquent location!
